In [1]:
import os
import tkinter as tk
from tkinter import ttk
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg

def add_expense():
    date = date_entry.get()
    category = category_entry.get()
    amount = amount_entry.get()

    if date and category and amount:
        with open("expenses.txt", "a") as file:
            file.write(f"{date},{category},{amount}\n")
        status_label.config(text="Expense added successfully!", fg="green")
        date_entry.delete(0, tk.END)
        category_entry.delete(0, tk.END)
        amount_entry.delete(0, tk.END)
        view_expenses()
    else:
        status_label.config(text="Please fill all the fields!", fg="red")

def delete_expense():
    selected_item = expenses_tree.selection()
    if selected_item:
        item_text = expenses_tree.item(selected_item, "values")
        date, category, amount = item_text
        with open("expenses.txt", "r") as file:
            lines = file.readlines()
        with open("expenses.txt", "w") as file:
            for line in lines:
                if line.strip() != f"{date},{category},{amount}":
                    file.write(line)
        status_label.config(text="Expense deleted successfully!", fg="green")
        view_expenses()
    else:
        status_label.config(text="Please select an expense to delete!", fg="red")

def view_expenses():
    global expenses_tree
    if os.path.exists("expenses.txt"):
        total_expense = 0
        expenses_tree.delete(*expenses_tree.get_children())
        with open("expenses.txt", "r") as file:
            for line in file:
                date, category, amount = line.strip().split(",")
                expenses_tree.insert("", tk.END, values=(date, category, amount))
                total_expense += float(amount)
        total_label.config(text=f"Total Expense: {total_expense:.2f}")
    else:
        total_label.config(text="No expenses recorded.")
        expenses_tree.delete(*expenses_tree.get_children())

def visualize_expenses():
    if os.path.exists("expenses.txt"):
        dates = []
        amounts = []
        with open("expenses.txt", "r") as file:
            for line in file:
                date, _, amount = line.strip().split(",")
                dates.append(date)
                amounts.append(float(amount))
        
        # Get the selected graph type
        graph_type = graph_type_var.get()
        
        # Create the selected type of graph
        if graph_type == "Line":
            plt.plot(dates, amounts, marker='o', linestyle='-')
        elif graph_type == "Bar":
            plt.bar(dates, amounts)
        
        plt.title('Expense Variation Over Time')
        plt.xlabel('Date')
        plt.ylabel('montant')
        plt.xticks(rotation=45)
        plt.tight_layout()

        # Convert the plot to an image file (PNG)
        plt.savefig("plot.png")

        # Create a Tkinter PhotoImage from the image file
        image = tk.PhotoImage(file="plot.png")

        # Display the image on the canvas
        canvas.create_image(0, 0, anchor="nw", image=image)
        canvas.image = image  # To prevent garbage collection

# Create the main application window
root = tk.Tk()
root.title("Expense Tracker")

# Set the initial size of the window
root.geometry("800x600")

# Create a notebook (tabbed interface)
notebook = ttk.Notebook(root)
notebook.pack(fill='both', expand=True)

# Create the first tab for adding/viewing expenses
expenses_tab = ttk.Frame(notebook)
notebook.add(expenses_tab, text='Expenses')

# Create labels and entries for adding expenses
date_label = tk.Label(expenses_tab, text="Date (YYYY-MM-DD):")
date_label.grid(row=0, column=0, padx=5, pady=5)
date_entry = tk.Entry(expenses_tab)
date_entry.grid(row=0, column=1, padx=5, pady=5)

amount_label = tk.Label(expenses_tab, text="Montant:")
amount_label.grid(row=2, column=0, padx=5, pady=5)
amount_entry = tk.Entry(expenses_tab)
amount_entry.grid(row=2, column=1, padx=5, pady=5)

category_label = tk.Label(expenses_tab, text="Description:")
category_label.grid(row=1, column=0, padx=5, pady=5)
category_entry = tk.Entry(expenses_tab)
category_entry.grid(row=1, column=1, padx=5, pady=5)

add_button = tk.Button(expenses_tab, text="Add Expense", command=add_expense)
add_button.grid(row=3, column=0, columnspan=2, padx=5, pady=10)

# Create a treeview to display expenses
columns = ("Date", "Montant", "Description")
expenses_tree = ttk.Treeview(expenses_tab, columns=columns, show="headings")
expenses_tree.heading("Date", text="Date")
expenses_tree.heading("Montant", text="Montant")
expenses_tree.heading("Description", text="Description")
expenses_tree.grid(row=4, column=0, columnspan=3, padx=5, pady=5)

# Create a label to display the total expense
total_label = tk.Label(expenses_tab, text="")
total_label.grid(row=5, column=0, columnspan=2, padx=5, pady=5)

# Create a label to show the status of expense addition and deletion
status_label = tk.Label(expenses_tab, text="", fg="green")
status_label.grid(row=6, column=0, columnspan=2, padx=5, pady=5)

# Create buttons to view and delete expenses
view_button = tk.Button(expenses_tab, text="View Expenses", command=view_expenses)
view_button.grid(row=7, column=0, padx=5, pady=10)

delete_button = tk.Button(expenses_tab, text="Delete Expense", command=delete_expense)
delete_button.grid(row=7, column=1, padx=5, pady=10)

# Check if the 'expenses.txt' file exists; create it if it doesn't
if not os.path.exists("expenses.txt"):
    with open("expenses.txt", "w"):
        pass

# Display existing expenses on application start
view_expenses()

# Create the second tab for visualizing expenses
visualization_tab = ttk.Frame(notebook)
notebook.add(visualization_tab, text='Visualization')

# Create a canvas for visualization
canvas = tk.Canvas(visualization_tab)
canvas.pack(fill='both', expand=True)

# Create a label for selecting graph type
graph_type_label = tk.Label(visualization_tab, text="Select Graph Type:")
graph_type_label.pack()

# Create a variable to store the selected graph type
graph_type_var = tk.StringVar(visualization_tab)
graph_type_var.set("Line")  # Default selection

# Create a dropdown menu for selecting graph type
graph_type_menu = ttk.OptionMenu(visualization_tab, graph_type_var, "Line", "Line", "Bar")
graph_type_menu.pack()

# Create a button for displaying the graph
visualize_button = tk.Button(visualization_tab, text="Visualize Expenses", command=visualize_expenses)
visualize_button.pack(pady=10)

root.mainloop()
